In [ ]:
#!pip install ultralytics
#!pip install gTTS
#!pip install pydub
import cv2
import torch
import os
from PIL import Image
from ultralytics import YOLO
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from gtts import gTTS
from IPython.display import Audio, display
from pydub import AudioSegment



folder_path = "/bin/Food_p_Photos"
# Model
model = torch.hub.load("ultralytics/yolov5", "yolov5s")
#model = torch.hub.load("ultralytics/yolov8", "yolov8l")  # Replace "yolov8s" with other versions like "yolov8m" or "yolov8l" if needed


#model = YOLO("yolo11n.pt")

# Images
'''for f in "zidane.jpg", "bus.jpg":
    #torch.hub.download_url_to_file("https://ultralytics.com/images/" + f, f)  # download 2 images
im1 = Image.open("zidane.jpg")  # PIL image
im2 = cv2.imread("bus.jpg")[..., ::-1]  # OpenCV image (BGR to RGB)'''


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith((".jpg", ".jpeg", ".png", ".bmp")):
            img = Image.open(os.path.join(folder, filename))
            images.append(img)
    return images

images = load_images_from_folder(folder_path)
#print('These are the images',images)
# Inference
df_image = pd.DataFrame()
df_counting=pd.DataFrame()
df_cnt_temp=pd.DataFrame()
#results = model("/bin/Food_p_Photos/fruit3.jpg")  # batch of images
i=0;

for image in images:
    i=i+1
    results = model(image)
    df_image=results.pandas().xyxy[0]  # im1 predictions (pandas)
    df_image['name_tra'] = df_image['name']
    df_image.loc[df_image['confidence'] <0.5, 'name_tra'] = 'unknown'
    counts = df_image['name_tra'].value_counts()
    df_cnt_temp=pd.DataFrame(counts)
    df_cnt_temp.reset_index(inplace=True)
    df_cnt_temp.columns.values[1] = 'Day ',i
   #df_cnt_temp=df_cnt_temp.transpose()
    if(i>1):
        df_counting = pd.merge(df_counting, df_cnt_temp, on='name_tra', how='outer')
    else:
        df_counting=df_cnt_temp
df_counting.fillna(0, inplace=True)
print(df_counting)

    #df_counting[('Day ',i)]=counts[:]

textst='Today we have only'
print(textst)
testarr1=[]
for row in range(len(df_cnt_temp)):
    testarr1.append(str(df_cnt_temp.iloc[row, 1]))
    testarr1.append(df_cnt_temp.iloc[row, 0])
    print(df_cnt_temp.iloc[row, 1],' ',df_cnt_temp.iloc[row, 0])
textst2='in the pantry'
print(textst2)

textst3='We are missing'
print(textst3)
testarr2=[]
for row in range(len(df_counting)):
    if(df_counting.iloc[row, (len(df_counting.columns)-1)]==0):
        if(df_counting.iloc[row, 0]!='unknown'):
            testarr2.append(df_counting.iloc[row, 0])
            print(df_counting.iloc[row, 0])
textst4='in the pantry'
print(textst4)

def generate_audio_segment(text, lang="en"):
    tts = gTTS(text=text, lang=lang)
    tts.save("temp.mp3")
    return AudioSegment.from_file("temp.mp3", format="mp3")

combined_audio = AudioSegment.empty()  # Start with an empty audio segment
segment = generate_audio_segment(textst)
combined_audio += segment
for text in testarr1:
    segment = generate_audio_segment(text)
    combined_audio += segment  # Append the audio segment
segment = generate_audio_segment(textst2)
combined_audio += segment

segment = generate_audio_segment(textst3)
combined_audio += segment
for text in testarr2:
    segment = generate_audio_segment(text)
    combined_audio += segment  # Append the audio segment
segment = generate_audio_segment(textst4)
combined_audio += segment
# Export the combined audio to a single file
combined_audio.export("combined_audio.mp3", format="mp3")

#Playing the saved audio file
display(Audio("combined_audio.mp3", autoplay=True))
